In [6]:
import os
import requests
import zipfile
from pathlib import Path
import pandas as pd

# Create the directory if it doesn't exist
data_dir = Path("../data/sus")
data_dir.mkdir(parents=True, exist_ok=True)

# Base URL pattern
base_url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/BPS/csv/{year}.csv.zip"

# Years to download
years = range(2020, 2025)  # 2020 to 2024 inclusive

for year in years:
    print(f"\n--- Processing year {year} ---")
    
    # URL for the current year
    url = base_url.format(year=year)
    
    try:
        # Download the file
        print(f"Downloading {year}.csv.zip...")
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Save the zip file
        zip_path = data_dir / f"{year}.csv.zip"
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        
        print(f"File downloaded and saved to: {zip_path}")
        
        # Extract the zip file
        print(f"Extracting {year}.csv.zip...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        
        # Delete the zip file after extraction
        print(f"Deleting {year}.csv.zip...")
        zip_path.unlink()
        
        # Find the extracted CSV file and fix encoding
        csv_file = data_dir / f"{year}.csv"
        if csv_file.exists():
            print(f"Converting encoding for {year}.csv...")
            # Try different encodings to handle accents properly
            for encoding in ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']:
                try:
                    df = pd.read_csv(csv_file, encoding=encoding)
                    # Save as UTF-8 to preserve accents
                    df.to_csv(csv_file, encoding='utf-8', index=False)
                    print(f"Successfully converted {year}.csv with {encoding} encoding!")
                    break
                except UnicodeDecodeError:
                    continue
            else:
                print(f"Warning: Could not determine encoding for {year}.csv")
        
        print(f"Processing completed for {year}!")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {year}: {e}")
    except zipfile.BadZipFile as e:
        print(f"Error extracting {year}: {e}")
    except Exception as e:
        print(f"Error processing {year}: {e}")

print(f"\nAll downloads completed!")

# List the contents of the directory to verify
print(f"\nContents of {data_dir}:")
for item in sorted(data_dir.iterdir()):
    print(f"  {item.name}")


--- Processing year 2020 ---
File downloaded and saved to: ..\data\sus\2020.csv.zip
Extracting 2020.csv.zip...
Deleting 2020.csv.zip...
Converting encoding for 2020.csv...
Error processing 2020: Error tokenizing data. C error: Expected 6 fields in line 28, saw 7


--- Processing year 2021 ---
File downloaded and saved to: ..\data\sus\2021.csv.zip
Extracting 2021.csv.zip...
Deleting 2021.csv.zip...
Converting encoding for 2021.csv...
Error processing 2021: Error tokenizing data. C error: Expected 4 fields in line 17, saw 5


--- Processing year 2022 ---
File downloaded and saved to: ..\data\sus\2022.csv.zip
Extracting 2022.csv.zip...
Deleting 2022.csv.zip...
Converting encoding for 2022.csv...
Error processing 2022: Error tokenizing data. C error: Expected 3 fields in line 4, saw 9


--- Processing year 2023 ---
File downloaded and saved to: ..\data\sus\2023.csv.zip
Extracting 2023.csv.zip...
Deleting 2023.csv.zip...
Converting encoding for 2023.csv...
Error processing 2023: Error toke

In [1]:
# Vamos analisar e corrigir o problema de encoding
import chardet
import os
import pandas as pd
from pathlib import Path

# Primeiro, definir o data_dir novamente após o restart do kernel
data_dir = Path("../data/sus")

# Verificar o encoding detectado do arquivo 2024.csv
csv_path = data_dir / "2024.csv"

if csv_path.exists():
    # Detectar encoding do arquivo
    with open(csv_path, 'rb') as f:
        raw_data = f.read(50000)  # Ler primeiros 50KB para melhor detecção
        detected = chardet.detect(raw_data)
        print(f"Encoding detectado pelo chardet: {detected}")
    
    # Tentar ler com diferentes encodings e mostrar uma amostra
    encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'windows-1252', 'cp850']
    
    for encoding in encodings_to_try:
        try:
            df_sample = pd.read_csv(csv_path, encoding=encoding, nrows=2, sep=';')
            print(f"\n--- Testando encoding: {encoding} ---")
            # Verificar se temos a coluna correta
            if 'Nome Instituição' in df_sample.columns:
                print("✓ Separador correto (;) e encoding funcionando")
                print("Colunas encontradas:", df_sample.columns.tolist()[:3])
                print("Amostra da primeira linha:")
                print(f"  Nome Instituição: {df_sample.iloc[0]['Nome Instituição']}")
                print(f"  Município: {df_sample.iloc[0]['Município Instituição'] if 'Município Instituição' in df_sample.columns else 'N/A'}")
                break
            else:
                print(f"❌ Colunas encontradas: {df_sample.columns.tolist()[:3]}")
        except Exception as e:
            print(f"❌ Erro com {encoding}: {str(e)[:100]}")
    
    print(f"\n--- Informações do arquivo ---")
    print(f"Tamanho: {csv_path.stat().st_size / (1024*1024):.2f} MB")
    
    # Verificar as primeiras linhas do arquivo raw
    print(f"\n--- Primeiras linhas do arquivo (raw) ---")
    with open(csv_path, 'rb') as f:
        first_lines = f.read(500)
        print(repr(first_lines))
        
else:
    print("Arquivo 2024.csv não encontrado")
    print("Arquivos disponíveis:")
    if data_dir.exists():
        for item in data_dir.iterdir():
            print(f"  {item.name}")

Encoding detectado pelo chardet: {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
❌ Erro com utf-8: 'utf-8' codec can't decode byte 0xe7 in position 13: invalid continuation byte

--- Testando encoding: latin-1 ---
✓ Separador correto (;) e encoding funcionando
Colunas encontradas: ['Nome Instituição', 'CNPJ Instituição', 'Município Instituição']
Amostra da primeira linha:
  Nome Instituição: FUNDO MUNICIPAL DE SAUDE
  Município: MURICI

--- Informações do arquivo ---
Tamanho: 7.62 MB

--- Primeiras linhas do arquivo (raw) ---
b'Nome Institui\xe7\xe3o;CNPJ Institui\xe7\xe3o;Munic\xedpio Institui\xe7\xe3o;UF;Compra;Inser\xe7\xe3o;Modalidade da Compra;C\xf3digo BR;Descri\xe7\xe3o CATMAT;Unidade Fornecimento;Gen\xe9rico;ANVISA;CNPJ Fornecedor;Fornecedor;CNPJ Fabricante;Fabricante;Qtd Itens Comprados;Pre\xe7o Unit\xe1rio;Pre\xe7o Total\r\nFUNDO MUNICIPAL DE SAUDE;11.120.699/0001-40;MURICI;AL;45292;45513;Preg\xe3o;365246;EXTRATO, TIPO:EXTRATO GLIC\xd3LICO, NOME COMUM:BABOSA, N

In [2]:
# VERSÃO CORRIGIDA: Download e processamento com encoding adequado
import os
import requests
import zipfile
from pathlib import Path
import pandas as pd
import chardet

def detect_and_convert_encoding(file_path):
    """
    Detecta o encoding do arquivo e converte para UTF-8 se necessário
    """
    # Detectar encoding
    with open(file_path, 'rb') as f:
        raw_data = f.read(100000)  # Ler 100KB para boa detecção
        detected = chardet.detect(raw_data)
        detected_encoding = detected['encoding']
        confidence = detected['confidence']
    
    print(f"  Encoding detectado: {detected_encoding} (confiança: {confidence:.2f})")
    
    # Lista de encodings para tentar, priorizando o detectado
    encodings_to_try = [detected_encoding, 'latin-1', 'cp1252', 'iso-8859-1', 'windows-1252']
    
    # Remover None e duplicatas, mantendo ordem
    encodings_to_try = list(dict.fromkeys([enc for enc in encodings_to_try if enc]))
    
    for encoding in encodings_to_try:
        try:
            # Tentar ler o arquivo
            df = pd.read_csv(file_path, encoding=encoding, sep=';')
            
            # Verificar se o arquivo foi lido corretamente
            if len(df.columns) > 10 and 'Nome Instituição' in df.columns:
                print(f"  ✓ Sucesso com encoding: {encoding}")
                
                # Salvar como UTF-8
                df.to_csv(file_path, encoding='utf-8', sep=';', index=False)
                print(f"  ✓ Arquivo convertido para UTF-8")
                
                return True, encoding
                
        except Exception as e:
            print(f"  ❌ Falha com {encoding}: {str(e)[:50]}...")
            continue
    
    return False, None

# Recriar diretório se necessário
data_dir = Path("../data/sus")
data_dir.mkdir(parents=True, exist_ok=True)

# Base URL pattern
base_url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/BPS/csv/{year}.csv.zip"

# Anos para download
years = range(2020, 2025)  # 2020 to 2024 inclusive

print("=== DOWNLOAD E CONVERSÃO CORRIGIDA ===\n")

for year in years:
    print(f"--- Processando ano {year} ---")
    
    # URL para o ano atual
    url = base_url.format(year=year)
    
    try:
        # Verificar se já existe e está ok
        csv_file = data_dir / f"{year}.csv"
        if csv_file.exists():
            print(f"Arquivo {year}.csv já existe. Verificando encoding...")
            
            # Tentar ler uma amostra para verificar se está ok
            try:
                test_df = pd.read_csv(csv_file, encoding='utf-8', sep=';', nrows=5)
                if 'Nome Instituição' in test_df.columns and len(test_df.columns) > 10:
                    print(f"✓ Arquivo {year}.csv já está correto!")
                    continue
            except:
                print(f"Arquivo {year}.csv existe mas precisa ser corrigido...")
        
        # Download do arquivo
        print(f"Baixando {year}.csv.zip...")
        response = requests.get(url)
        response.raise_for_status()
        
        # Salvar o arquivo zip
        zip_path = data_dir / f"{year}.csv.zip"
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        
        print(f"  Arquivo baixado: {zip_path}")
        
        # Extrair o arquivo zip
        print(f"Extraindo {year}.csv.zip...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        
        # Deletar o arquivo zip
        zip_path.unlink()
        print(f"  Arquivo zip removido")
        
        # Processar encoding do CSV extraído
        if csv_file.exists():
            print(f"Processando encoding de {year}.csv...")
            success, used_encoding = detect_and_convert_encoding(csv_file)
            
            if success:
                print(f"✓ {year}.csv processado com sucesso usando {used_encoding}")
            else:
                print(f"❌ Não foi possível processar {year}.csv corretamente")
        
        print(f"✓ Processamento concluído para {year}!\n")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erro no download de {year}: {e}")
    except zipfile.BadZipFile as e:
        print(f"❌ Erro na extração de {year}: {e}")
    except Exception as e:
        print(f"❌ Erro no processamento de {year}: {e}")

print("=== DOWNLOAD CONCLUÍDO ===\n")

# Verificar conteúdo do diretório
print("Arquivos no diretório:")
for item in sorted(data_dir.iterdir()):
    if item.is_file():
        size_mb = item.stat().st_size / (1024*1024)
        print(f"  {item.name} ({size_mb:.2f} MB)")

=== DOWNLOAD E CONVERSÃO CORRIGIDA ===

--- Processando ano 2020 ---
Arquivo 2020.csv já existe. Verificando encoding...
Arquivo 2020.csv existe mas precisa ser corrigido...
Baixando 2020.csv.zip...
  Arquivo baixado: ..\data\sus\2020.csv.zip
Extraindo 2020.csv.zip...
  Arquivo zip removido
Processando encoding de 2020.csv...
  Encoding detectado: MacRoman (confiança: 0.73)
❌ Não foi possível processar 2020.csv corretamente
✓ Processamento concluído para 2020!

--- Processando ano 2021 ---
Arquivo 2021.csv já existe. Verificando encoding...
Arquivo 2021.csv existe mas precisa ser corrigido...
Baixando 2021.csv.zip...
  Arquivo baixado: ..\data\sus\2021.csv.zip
Extraindo 2021.csv.zip...
  Arquivo zip removido
Processando encoding de 2021.csv...
  Encoding detectado: ISO-8859-1 (confiança: 0.73)
❌ Não foi possível processar 2021.csv corretamente
✓ Processamento concluído para 2021!

--- Processando ano 2022 ---
Arquivo 2022.csv já existe. Verificando encoding...
Arquivo 2022.csv existe m

In [3]:
# TESTE FINAL: Verificar se os acentos estão sendo lidos corretamente
print("=== TESTE DE LEITURA COM ACENTOS ===\n")

# Testar leitura do arquivo 2024.csv
csv_path = data_dir / "2024.csv"

if csv_path.exists():
    try:
        # Ler com UTF-8
        df = pd.read_csv(csv_path, encoding='utf-8', sep=';')
        
        print(f"Arquivo carregado com sucesso!")
        print(f"Dimensões: {df.shape}")
        print(f"Colunas: {len(df.columns)}")
        
        print(f"\n--- Teste de acentos ---")
        print("Primeiras 3 colunas:")
        for col in df.columns[:3]:
            print(f"  {col}")
        
        print(f"\nPrimeiras 5 linhas da coluna 'Nome Instituição':")
        for i, nome in enumerate(df['Nome Instituição'].head()):
            print(f"  {i+1}. {nome}")
        
        print(f"\nPrimeiras 5 linhas da coluna 'Município Instituição':")
        for i, municipio in enumerate(df['Município Instituição'].head()):
            print(f"  {i+1}. {municipio}")
            
        print(f"\nPrimeiras 5 linhas da coluna 'Descrição CATMAT':")
        for i, desc in enumerate(df['Descrição CATMAT'].head()):
            print(f"  {i+1}. {desc[:100]}...")  # Mostrar apenas os primeiros 100 caracteres
        
        # Verificar se temos acentos
        nome_com_acento = df['Nome Instituição'].str.contains('ção|ção|ão|ões|ções|ê|á|é|í|ó|ú|â|ã|õ|ç', na=False).any()
        municipio_com_acento = df['Município Instituição'].str.contains('ção|ção|ão|ões|ções|ê|á|é|í|ó|ú|â|ã|õ|ç', na=False).any()
        
        print(f"\n--- Verificação de acentos ---")
        print(f"Acentos em 'Nome Instituição': {'✓ SIM' if nome_com_acento else '❌ NÃO'}")
        print(f"Acentos em 'Município Instituição': {'✓ SIM' if municipio_com_acento else '❌ NÃO'}")
        
        if nome_com_acento or municipio_com_acento:
            print("\n🎉 SUCESSO! Os acentos estão sendo lidos corretamente!")
        else:
            print("\n⚠️  Os acentos ainda não estão sendo detectados adequadamente.")
            
    except Exception as e:
        print(f"❌ Erro ao ler o arquivo: {e}")
else:
    print("❌ Arquivo 2024.csv não encontrado")

=== TESTE DE LEITURA COM ACENTOS ===

Arquivo carregado com sucesso!
Dimensões: (24635, 19)
Colunas: 19

--- Teste de acentos ---
Primeiras 3 colunas:
  Nome Instituição
  CNPJ Instituição
  Município Instituição

Primeiras 5 linhas da coluna 'Nome Instituição':
  1. FUNDO MUNICIPAL DE SAUDE
  2. FUNDO MUNICIPAL DE SAUDE
  3. FUNDO MUNICIPAL DE SAUDE
  4. FUNDO MUNICIPAL DE SAUDE
  5. FUNDO MUNICIPAL DE SAUDE

Primeiras 5 linhas da coluna 'Município Instituição':
  1. MURICI
  2. MURICI
  3. MURICI
  4. MURICI
  5. MURICI

Primeiras 5 linhas da coluna 'Descrição CATMAT':
  1. EXTRATO, TIPO:EXTRATO GLICÓLICO, NOME COMUM:BABOSA, NOME BOTÂNICO:ALOE VERA L., ASPECTO FÍSICO:LÍQUI...
  2. OLANZAPINA, DOSAGEM:5 MG...
  3. VALPROATO DE SÓDIO, CONCENTRAÇÃO:500 MG...
  4. NORETISTERONA, CONCENTRAÇAO:0,35 MG, CARACTERÍSTICAS ADICIONAIS:EM BLISTER CALENDÁRIO...
  5. EQUIPO ESPECIAL, APLICAÇÃO:P/ INJEÇÃO GÁS CARBÔNICO CO2 - VIA SC, NÚMERO VIAS:VIA ÚNICA, MATERIAL:TU...

--- Verificação de acentos -

In [4]:
# FUNÇÃO UTILITÁRIA: Carregar dados de qualquer ano
def carregar_dados_sus(ano, sample_size=None):
    """
    Carrega dados do SUS para um ano específico
    
    Args:
        ano (int): Ano dos dados (2020-2024)
        sample_size (int, optional): Número de linhas para carregar (None = todas)
    
    Returns:
        pandas.DataFrame: Dados carregados
    """
    csv_path = data_dir / f"{ano}.csv"
    
    if not csv_path.exists():
        raise FileNotFoundError(f"Arquivo {ano}.csv não encontrado. Execute o download primeiro.")
    
    try:
        if sample_size:
            df = pd.read_csv(csv_path, encoding='utf-8', sep=';', nrows=sample_size)
        else:
            df = pd.read_csv(csv_path, encoding='utf-8', sep=';')
        
        print(f"✓ Dados de {ano} carregados: {df.shape[0]:,} registros, {df.shape[1]} colunas")
        return df
        
    except Exception as e:
        raise Exception(f"Erro ao carregar dados de {ano}: {e}")

def resumo_dados_sus(ano):
    """
    Mostra um resumo dos dados do SUS para um ano
    """
    df = carregar_dados_sus(ano, sample_size=1000)  # Carregar amostra para resumo
    
    print(f"\n=== RESUMO DOS DADOS DE {ano} ===")
    print(f"Colunas disponíveis:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\nEstatísticas básicas:")
    print(f"  • Instituições únicas: {df['Nome Instituição'].nunique():,}")
    print(f"  • Municípios únicos: {df['Município Instituição'].nunique():,}")
    print(f"  • UFs únicas: {df['UF'].nunique()}")
    print(f"  • Fornecedores únicos: {df['Fornecedor'].nunique():,}")
    
    # Converter preço total para numérico para estatísticas
    df['Preço Total'] = pd.to_numeric(df['Preço Total'].astype(str).str.replace(',', '.'), errors='coerce')
    
    print(f"  • Valor total (amostra): R$ {df['Preço Total'].sum():,.2f}")
    print(f"  • Valor médio por item: R$ {df['Preço Total'].mean():,.2f}")
    
    return df

# Exemplo de uso
print("=== FUNÇÕES CRIADAS ===")
print("✓ carregar_dados_sus(ano, sample_size=None)")
print("✓ resumo_dados_sus(ano)")
print("\nExemplo de uso:")
print("  df_2024 = carregar_dados_sus(2024)")
print("  resumo_dados_sus(2024)")

# Testar com dados de 2024
try:
    resumo_2024 = resumo_dados_sus(2024)
except Exception as e:
    print(f"Erro no teste: {e}")

=== FUNÇÕES CRIADAS ===
✓ carregar_dados_sus(ano, sample_size=None)
✓ resumo_dados_sus(ano)

Exemplo de uso:
  df_2024 = carregar_dados_sus(2024)
  resumo_dados_sus(2024)
✓ Dados de 2024 carregados: 1,000 registros, 19 colunas

=== RESUMO DOS DADOS DE 2024 ===
Colunas disponíveis:
   1. Nome Instituição
   2. CNPJ Instituição
   3. Município Instituição
   4. UF
   5. Compra
   6. Inserção
   7. Modalidade da Compra
   8. Código BR
   9. Descrição CATMAT
  10. Unidade Fornecimento
  11. Genérico
  12. ANVISA
  13. CNPJ Fornecedor
  14. Fornecedor
  15. CNPJ Fabricante
  16. Fabricante
  17. Qtd Itens Comprados
  18. Preço Unitário
  19. Preço Total

Estatísticas básicas:
  • Instituições únicas: 5
  • Municípios únicos: 5
  • UFs únicas: 5
  • Fornecedores únicos: 12
  • Valor total (amostra): R$ 17,749,162.65
  • Valor médio por item: R$ 17,749.16
